# Import Libraries

In [50]:
import os
import re

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load corpus

### Load Positives

In [51]:
train_tweets, train_labels = [], []

pos = os.getcwd() + '/corpus/arabic_tweets/pos/'  # Replace with the actual directory path

# Iterate over each file in the directory
for filename in os.listdir(pos):
    if filename.endswith('.txt'):  # Select only text files
        file_path = os.path.join(pos, filename)
        with open(file_path, 'r', encoding='utf-8-sig') as file:
            file_content = file.read()
            train_tweets.append(file_content)
            train_labels.append("positive")

### Load Negatives

In [52]:
# Get the txt file negative tweet
neg = os.getcwd() + '/corpus/arabic_tweets/neg/'  # Replace with the actual directory path

# Iterate over each file in the directory
for filename in os.listdir(neg):
    if filename.endswith('.txt'):  # Select only text files
        file_path = os.path.join(neg, filename)
        with open(file_path, 'r', encoding='utf-8-sig') as file:
            file_content = file.read()
            train_tweets.append(file_content)
            train_labels.append("negative")

### Build a dataframe

In [53]:
train_dic = {
    'Tweets' : train_tweets,
    'Labels' : train_labels
}

train_corpus = pd.DataFrame(train_dic)
train_corpus.head()

,Tweets,Labels
0,دامك مع #غناتي ، فالك طيب 👍\n,positive
1,على الفطرة السليمه.. الله يعطيه الصحة والعافية...,positive
2,📷 مشجع هلالي ينبذ العنصرية ب لافته أعدها.\n,positive
3,سبحان الله🌸 الحمدلله 💮 لا اله الا الله 🌿 الله ...,positive
4,مشاركتي في مبادراتكم الجميلة فوز وسعادة 💞\n,positive


# EDA

##### Explore your dataset

In [54]:
train_corpus

,Tweets,Labels
0,دامك مع #غناتي ، فالك طيب 👍\n,positive
1,على الفطرة السليمه.. الله يعطيه الصحة والعافية...,positive
2,📷 مشجع هلالي ينبذ العنصرية ب لافته أعدها.\n,positive
3,سبحان الله🌸 الحمدلله 💮 لا اله الا الله 🌿 الله ...,positive
4,مشاركتي في مبادراتكم الجميلة فوز وسعادة 💞\n,positive
...,...,...
58746,تو اتابع الملخص اياكس شيء عظيم يخوان والله افت...,negative
58747,يوم كامل وفقط أضفت فقرتين 😑\n,negative
58748,ما رح اتخلى عنك ❤ حتى لو بدي ازعل واتوجع منك 💔...,negative
58749,والله عيب .. البعض يحتاج سنوات ضوئية .. حتى يت...,negative


# Data Preprocessing

### Shuffle all rows

In [55]:
train_corpus = train_corpus.sample(frac=1).reset_index(drop=True)

In [56]:
train_corpus

,Tweets,Labels
0,مررت بكل ذلك وحدي ،، فلا تخبرني اننا أصدقاء 💔\n,negative
1,.♥ . ♥. ♥ صباح ♥ ♥الذاكرين♥ .♥ . ♥. أسأل الله ...,positive
2,بعين الله 😳 ازا الاجازة ما كانت يومين ما بتتسم...,negative
3,🏆 تتويج بطلا للدوري 🔻 هبوط الباطن والتسامح ⚽️ ...,positive
4,نختلف فى الميول ونختلف فى الأفكار ونختلف في أش...,positive
...,...,...
58746,🌸 طوينا الكثير من الصفحات في حياتنا لأننا أنتھ...,positive
58747,سبحان الله وبحمده سبحان الله العظيم ارح مسمعك 😴\n,negative
58748,ماعندي خبر 💔 بس قد صارتلي نفس المشكلة وانا موب...,negative
58749,بمناسبة فوز الهلال .. 💙 سحب على آيفون XR📱 رتوي...,positive


### Data cleaning

**Hint: remove URLs, Hashtags, alphanumeric characters, punctuation marks, stop words, extra spaces**

In [57]:
URL_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
hashtag_pattern = r"#\w+"
mention_pattern = r"@\w+"
alphanumeric_pattern = r"\w*\d\w*"
punctuation_pattern = r"[^\w\s]"
retweet_pattern = r"^RT[\s]+"

In [58]:
def load_stopwords(file_path):
    with open(file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
    return frozenset(stop_set)

def process_text(text, stop_words):
    # Remove URLs
    text = re.sub(URL_pattern, '', text)
    
    # Remove hashtags
    text = re.sub(hashtag_pattern, '', text)
    
    # Remove mention
    text = re.sub(mention_pattern, '', text)

    # Remove alphanumeric characters
    text = re.sub(alphanumeric_pattern, '', text)

    # Remove punctuation marks
    text = re.sub(punctuation_pattern, '', text)
    
    # Remove Retweet marks
    text = re.sub(retweet_pattern, '', text)

    # Remove stop words using the provided set
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    text = ' '.join(text.split())
    return text

#### Now Clean your text using above function or implement it from scrach

In [59]:
stop = load_stopwords('corpus/Stop_Words.txt')

In [60]:
for i in range(train_corpus.shape[0]):
    train_corpus.loc[i,'Tweets'] = process_text(train_corpus.loc[i,'Tweets'],stop)

In [61]:
train_corpus.reset_index(inplace=True,drop=True)
train_corpus

,Tweets,Labels
0,مررت بكل وحدي فلا تخبرني اننا أصدقاء,negative
1,الذاكرين أسأل الله أن يبشرك بما يسرك ويكف عنك ...,positive
2,بعين الله ازا الاجازة يومين بتتسمى اجازة,negative
3,تتويج بطلا للدوري هبوط الباطن والتسامح تقرير خ...,positive
4,نختلف الميول ونختلف الأفكار ونختلف أشياء كثيرا,positive
...,...,...
58746,طوينا الكثير الصفحات حياتنا لأننا أنتھينا قراء...,positive
58747,سبحان الله وبحمده سبحان الله العظيم ارح مسمعك,negative
58748,ماعندي خبر بس صارتلي نفس المشكلة وانا موبايلي ...,negative
58749,بمناسبة فوز الهلال سحب آيفون XR رتويت وتابع ال...,positive


#### Extra: you could do stemming or lemmatization before training

# Split data to train and test

In [62]:
split_ratio = 0.8 
split_index = int(split_ratio * len(train_corpus))
x_train, y_train = train_corpus.loc[:split_index,'Tweets'], train_corpus.loc[:split_index,'Labels']
x_test, y_test = train_corpus.loc[split_index:,'Tweets'], train_corpus.loc[split_index:,'Labels']

In [63]:
x_test

47000                                 جزاك الله احسن الجزا
47001    هه طظ الدنيا ياجدع والنعمه انت رايق ولافارق مع...
47002                                                انشهد
47003    بمناسبة فوز الهلال سحب آيفون XR رتويت وتابع ال...
47004                     الحمدالله السلامة ابو عبد المحسن
                               ...                        
58746    طوينا الكثير الصفحات حياتنا لأننا أنتھينا قراء...
58747        سبحان الله وبحمده سبحان الله العظيم ارح مسمعك
58748    ماعندي خبر بس صارتلي نفس المشكلة وانا موبايلي ...
58749    بمناسبة فوز الهلال سحب آيفون XR رتويت وتابع ال...
58750                         كيس الشعلان تتهمنا بالعنصرية
Name: Tweets, Length: 11751, dtype: object

# Tokenizer

In [64]:
token = Tokenizer(oov_token='<OOV>')
token.fit_on_texts(x_train)
word_index = token.word_index

# Text to sequence

In [65]:
sequ = token.texts_to_sequences(x_train)

# Pad sequence

In [66]:
max_sequ = max(len(seq) for seq in sequ)
paded = pad_sequences(sequ,maxlen=max_sequ,padding='post')

In [67]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

# RNN Model

In [68]:
model = Sequential()
model.add(Embedding(input_dim=len(token.word_index) + 1, output_dim=100, input_length=max_sequ))
model.add(SimpleRNN(units=100))
model.add(Dense(units=1, activation='sigmoid'))

In [69]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
model.fit(paded,y_train, epochs=10 ,batch_size=128)

Epoch 1/10
368/368 [==============================] - 9s 23ms/step - loss: 0.4970 - accuracy: 0.7213
Epoch 2/10
368/368 [==============================] - 9s 23ms/step - loss: 0.1573 - accuracy: 0.9362
Epoch 3/10
368/368 [==============================] - 10s 26ms/step - loss: 0.0731 - accuracy: 0.9709
Epoch 4/10
368/368 [==============================] - 9s 24ms/step - loss: 0.0564 - accuracy: 0.9769
Epoch 5/10
368/368 [==============================] - 9s 24ms/step - loss: 0.0481 - accuracy: 0.9788
Epoch 6/10
368/368 [==============================] - 8s 23ms/step - loss: 0.0434 - accuracy: 0.9807
Epoch 7/10
368/368 [==============================] - 8s 23ms/step - loss: 0.0424 - accuracy: 0.9803
Epoch 8/10
368/368 [==============================] - 9s 24ms/step - loss: 0.0394 - accuracy: 0.9824
Epoch 9/10
368/368 [==============================] - 10s 28ms/step - loss: 0.0387 - accuracy: 0.9824
Epoch 10/10
368/368 [==============================] - 8s 23ms/step - loss: 0.0367 - accu

# LSTM Model

In [71]:
model1 = Sequential()
model1.add(Embedding(input_dim=len(token.word_index) + 1, output_dim=100, input_length=max_sequ))
model1.add(LSTM(units=100))
model1.add(Dense(units=1, activation='sigmoid'))

In [72]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [73]:
model1.fit(paded,y_train, epochs=10 ,batch_size=128)

Epoch 1/10
368/368 [==============================] - 15s 40ms/step - loss: 0.5092 - accuracy: 0.7211
Epoch 2/10
368/368 [==============================] - 15s 41ms/step - loss: 0.2636 - accuracy: 0.8859
Epoch 3/10
368/368 [==============================] - 15s 41ms/step - loss: 0.1366 - accuracy: 0.9448
Epoch 4/10
368/368 [==============================] - 15s 41ms/step - loss: 0.0926 - accuracy: 0.9616
Epoch 5/10
368/368 [==============================] - 15s 41ms/step - loss: 0.0751 - accuracy: 0.9668
Epoch 6/10
368/368 [==============================] - 15s 42ms/step - loss: 0.0639 - accuracy: 0.9689
Epoch 7/10
368/368 [==============================] - 17s 46ms/step - loss: 0.0604 - accuracy: 0.9712
Epoch 8/10
368/368 [==============================] - 16s 42ms/step - loss: 0.0559 - accuracy: 0.9733
Epoch 9/10
368/368 [==============================] - 16s 42ms/step - loss: 0.0504 - accuracy: 0.9745
Epoch 10/10
368/368 [==============================] - 15s 41ms/step - loss: 0.048

# Evaulation and Comparsion

In [74]:
token = Tokenizer(oov_token='<OOV>')
token.fit_on_texts(x_test)
word_index = token.word_index

sequ = token.texts_to_sequences(x_test)

paded = pad_sequences(sequ,maxlen=max_sequ,padding='post')

In [75]:
label_encoder = LabelEncoder()
y_test = label_encoder.fit_transform(y_test)

In [76]:
model.evaluate(x=paded,y=y_test)

368/368 [==============================] - 1s 1ms/step - loss: 1.4607 - accuracy: 0.4971


[1.4607253074645996, 0.49706408381462097]

In [77]:
model1.evaluate(paded,y_test)

368/368 [==============================] - 1s 3ms/step - loss: 3.0862 - accuracy: 0.5226


[3.0861785411834717, 0.5225937962532043]